In [ ]:
## imports 
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# comment these out if you don't have plotnine--not essential here/only used once
import plotnine
from plotnine import *

## way to connect to mysql 
## if you need to install
## uncomment this line:
#! pip install mysql-connector-python
import mysql.connector

## print mult
## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## function to feed path name to load
## credentials
def load_creds(path: str):
    with open(path, 'r') as stream:
        try:
            creds = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return(creds)


# 0. Preliminary: define connection and read sample of data

In [ ]:
creds = load_creds("09_db_cred.yaml")

In [ ]:
## connect to the database
cnx = mysql.connector.connect(user=creds['practice_database']['user'], 
                            password=creds['practice_database']['password'],
                            port=creds['practice_database']['port'],
                            database= creds['practice_database']['database'],
                            host = creds['practice_database']['host'])
cnx

In [ ]:
## define a query to pull first 5 rows
## from caseinit table
sample_case_q = """ 
select 
    * 
from caseinit 
"""

In [ ]:
## feed read sql query the query
## and my database connection
read_sample_d = pd.read_sql_query(sample_case_q, cnx)

read_sample_d

# 1. Row and column filtering

In [ ]:
## 1.1 Pulling columns

In [ ]:
## select specific columns
pull_id_q = """
select 
    CASE_ID, 
    CASE_PARTICIPANT_ID
from caseinit
"""

id_d = pd.read_sql_query(pull_id_q,cnx)
id_d.head()

## 1.2 subsetting rows

In [ ]:
## select specific rows using where
pull_id_age_q = """
select 
    CASE_ID, 
    CASE_PARTICIPANT_ID, 
    AGE_AT_INCIDENT
from caseinit
where AGE_AT_INCIDENT > 40
"""

id_age = pd.read_sql_query(pull_id_age_q,
                cnx)
id_age.head()

In [ ]:
## select specific rows using IN
pull_id_r_q = """
select 
    CASE_ID, 
    CASE_PARTICIPANT_ID, 
    RACE
from caseinit
where RACE in ("Black", "HISPANIC")
"""

id_r = pd.read_sql_query(pull_id_r_q,
                cnx)
id_r.RACE.value_counts()

## 1.3 Creating new columns

In [ ]:
## create new col 
## using case when logic
compare_charge_q = """
SELECT *,
       CASE
         WHEN offense_category = updated_offense_category THEN 'Same offense'
         ELSE 'Diff offense'
       END AS charge_update
FROM   caseinit 
"""

compare_charge_d = pd.read_sql_query(compare_charge_q, cnx)

compare_charge_d.charge_update.value_counts()
samp = compare_charge_d.groupby('charge_update').sample(n = 2, random_state = 919)
samp[['OFFENSE_CATEGORY', 'UPDATED_OFFENSE_CATEGORY', 'charge_update']]

## 1.4 Code that deliberately throws error due to creating var in same step as row filter

In [ ]:
## create new col 
## using case when logic
## and then row filter
## throws error bc doesn't
## know that charge_update exists
compare_charge_filt_q = """
SELECT *,
       CASE
         WHEN offense_category = updated_offense_category THEN 'Same offense'
         ELSE 'Diff offense'
       END AS charge_update
FROM   caseinit
WHERE  charge_update = 'Diff offense' 
"""

compare_charge_filt_d = pd.read_sql_query(compare_charge_filt_q, cnx)


In [ ]:
## row filtering using not equal syntax
compare_charge_q = """
select 
    *
    from caseinit
where OFFENSE_CATEGORY <> UPDATED_OFFENSE_CATEGORY
"""

test = pd.read_sql_query(compare_charge_q, cnx)
test[['OFFENSE_CATEGORY', 'UPDATED_OFFENSE_CATEGORY']].head()

# 2. Subqueries and aggregation: one dataset

## 2.1 Row filtering using subquery

In [ ]:
## row filtering using subquery
compare_charge_sub_q = """
SELECT *
FROM   caseinit
       INNER JOIN (SELECT case_id    AS cid,
                          case_participant_id AS cpid,
                          CASE
                            WHEN offense_category = updated_offense_category
                          THEN
                            'Same offense'
                            ELSE 'Diff offense'
                          END                 AS charge_update
                   FROM   caseinit) AS tmp
               ON tmp.cid = caseinit.case_id
                  AND tmp.cpid = caseinit.case_participant_id
WHERE  charge_update = "diff offense" 
"""

diff_c = pd.read_sql_query(compare_charge_sub_q, cnx)
diff_c[['OFFENSE_CATEGORY', 'UPDATED_OFFENSE_CATEGORY', 'charge_update']].head()

## 2.2 Longer example: racial disparities in who's offered diversion

-  Find the five most common offenses in the `caseinit` table
- For those five most common offenses, find the percent of Black defendants whose cases are diverted and the percent of White defendants whose cases are diverted


In [ ]:
## top 5 offenses
t5 = """
SELECT updated_offense_category,
       Count(*) AS count_offense
FROM   caseinit
WHERE  race IN ( "black", "white" )
GROUP  BY updated_offense_category
ORDER  BY count_offense DESC
LIMIT  5 
"""


t5_d = pd.read_sql_query(t5, cnx)
t5_d

In [ ]:
## filter to rows where UPDATED_OFFENSE_CATEGORY
## is in the top 5
t5_rowfilt = """
SELECT *
FROM   caseinit
       INNER JOIN (SELECT updated_offense_category AS tmp_oc,
                          Count(*)                 AS count_offense
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category
                   ORDER  BY count_offense DESC
                   LIMIT  5) AS top5
               ON caseinit.updated_offense_category = top5.tmp_oc
WHERE  race IN ( "black", "white" ) 
"""

t5_d = pd.read_sql_query(t5_rowfilt, cnx)
t5_d.shape
t5_d.RACE.value_counts()
t5_d.UPDATED_OFFENSE_CATEGORY.value_counts()

In [ ]:
## get diversion rates by race for white and black
## defendants 
divert_compare = """
SELECT updated_offense_category,
       is_in_diversion,
       race,
       Count(*)               AS count_divert,
       Count(*) / count_group AS prop_divert
FROM   caseinit
       INNER JOIN (SELECT updated_offense_category AS tmp_oc,
                          race                     AS tmp_race,
                          Count(*)                 AS count_group
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category,
                             race) AS tmp
               ON tmp.tmp_race = caseinit.race
                  AND tmp.tmp_oc = caseinit.updated_offense_category
GROUP  BY updated_offense_category,
          race,
          is_in_diversion
ORDER  BY count_divert DESC 
"""

divert_compare_alloff = pd.read_sql_query(divert_compare, cnx)
divert_compare_alloff.head()


In [ ]:
## combine with the top 5 filtering code
divert_compare_t5 = """
SELECT updated_offense_category,
       is_in_diversion,
       race,
       Count(*)               AS count_divert,
       Count(*) / count_group AS prop_divert
FROM   caseinit
       INNER JOIN (SELECT updated_offense_category AS tmp_oc,
                          race                     AS tmp_race,
                          Count(*)                 AS count_group
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category,
                             race) AS tmp
               ON tmp.tmp_race = caseinit.race
                  AND tmp.tmp_oc = caseinit.updated_offense_category
       INNER JOIN (SELECT updated_offense_category AS tmp_oc_t5,
                          Count(*)                 AS count_offense
                   FROM   caseinit
                   WHERE  race IN ( "black", "white" )
                   GROUP  BY updated_offense_category
                   ORDER  BY count_offense DESC
                   LIMIT  5) AS top5
               ON caseinit.updated_offense_category = top5.tmp_oc_t5
WHERE  is_in_diversion = 'True'
GROUP  BY updated_offense_category,
          race,
          is_in_diversion 
"""

divert_compare_t5  = pd.read_sql_query(divert_compare_t5, cnx)
divert_compare_t5


# 3. Subqueries and aggregations: two datasets

## 3.1 Looking at structure of diversions dataset

In [ ]:
## define a query to pull first 5 rows
## from caseinit table
sample_div_q = """
SELECT *
FROM   divert
LIMIT  5 
"""

divert_res = pd.read_sql_query(sample_div_q, cnx)
divert_res.head()
divert_res.columns

## 3.2 Left joining certain cols from caseinit onto diversions

### Left join with everything from divert; certain cols from caseinit

In [ ]:
lj_q = """
SELECT divert.*,
       age_at_incident,
       law_enforcement_agency,
       incident_city
FROM   divert
       LEFT JOIN caseinit
              ON divert.case_id = caseinit.case_id
              AND divert.case_participant_id = caseinit.case_participant_id 
"""

In [ ]:
lj_d = pd.read_sql_query(lj_q, cnx)
lj_d.head()
lj_d.shape
lj_d.columns

In [ ]:
lj_dupfix_q = """
SELECT divert.*,
       age_at_incident,
       law_enforcement_agency,
       incident_city,
       caseinit.race AS caseinit_race
FROM   divert
       LEFT JOIN caseinit
              ON divert.case_id = caseinit.case_id
                 AND divert.case_participant_id = caseinit.case_participant_id 
"""

In [ ]:
lj_d_dupfix = pd.read_sql_query(lj_dupfix_q, cnx)

In [ ]:
## only diff is that in diversion dataset
## they have the category Latinx
## while in case initiations that's Hispanic or 
## White hispanic/latino
pd.crosstab(lj_d_dupfix.RACE, lj_d_dupfix.caseinit_race)

### Aliasing the tables in the join

In [ ]:
lj_alias_q = """
SELECT d.*,
       age_at_incident,
       law_enforcement_agency,
       incident_city,
       c.race AS caseinit_race
FROM   divert AS d
       LEFT JOIN caseinit AS c
              ON d.case_id = c.case_id
                 AND d.case_participant_id = c.case_participant_id 
"""

In [ ]:
lj_alias_d = pd.read_sql_query(lj_alias_q, cnx)

# 4. Combining agg and join

Task: among the cases that are diverted, for each of the charges (`UPDATED_OFFENSE_CATEGORY`) in the case initiations, find the percentage of defendants with that charge going to each `DIVERSION_PROGRAM`

### First part: count of offenses by diversion program

In [ ]:
offenses_byprogram = """
SELECT Count(*) AS count_offenses_byprogram,
       updated_offense_category,
       diversion_program
FROM   divert
       INNER JOIN caseinit
               ON divert.case_id = caseinit.case_id
                  AND divert.case_participant_id = caseinit.case_participant_id
GROUP  BY updated_offense_category,
          diversion_program
ORDER  BY count_offenses_byprogram DESC 
"""

In [ ]:
cp = pd.read_sql_query(offenses_byprogram, cnx)
cp.head()

cp[cp.UPDATED_OFFENSE_CATEGORY == "Narcotics"]

### Second part: count of offenses total

In [ ]:
offenses_total = """
SELECT Count(*) AS count_offenses_total,
       updated_offense_category
FROM   divert AS d
       INNER JOIN caseinit AS c
               ON d.case_id = c.case_id
                  AND d.case_participant_id = c.case_participant_id
GROUP  BY updated_offense_category
ORDER  BY count_offenses_total DESC 
"""

In [ ]:
c = pd.read_sql_query(offenses_total, cnx)
c.head()

## check that total matches sum from first query- see both are 520 
cp.count_offenses_byprogram[cp.UPDATED_OFFENSE_CATEGORY == "Narcotics"].sum()

### Combining into one query 

Put the numerator into a subquery and do the proportions in the outer query. We add the second inner join to make sure that when we get the denominator from the outer part, we're restricting to defendants sent to diversion

In [ ]:
offenses_prop = """
SELECT count_offenses_byprogram / Count(*) AS prop_offenses_byprogram,
       caseinit.updated_offense_category,
       diversion_program
FROM   caseinit
       INNER JOIN (SELECT Count(*) AS count_offenses_byprogram,
                          updated_offense_category,
                          diversion_program
                   FROM   divert
                          INNER JOIN caseinit
                                  ON divert.case_id = caseinit.case_id
                                     AND divert.case_participant_id =
                                         caseinit.case_participant_id
                   GROUP  BY updated_offense_category,
                             diversion_program) AS num
               ON num.updated_offense_category =
                  caseinit.updated_offense_category
       INNER JOIN (SELECT case_id             AS cid,
                          case_participant_id AS cpid
                   FROM   divert) AS ppl_divert
               ON caseinit.case_id = ppl_divert.cid
                  AND caseinit.case_participant_id = ppl_divert.cpid
GROUP  BY caseinit.updated_offense_category,
          diversion_program 
"""

## note: check the denom

In [ ]:
cd_prop = pd.read_sql_query(offenses_prop, cnx)



In [ ]:
cd_prop.head()
cd_prop[cd_prop.UPDATED_OFFENSE_CATEGORY == "Narcotics"]

## confirm with above counts- see ddpp matches up! :)
prop_ddpp = cp.count_offenses_byprogram[(cp.UPDATED_OFFENSE_CATEGORY == "Narcotics") &
                                       (cp.DIVERSION_PROGRAM == "DDPP")].iloc[0]/ \
            c.count_offenses_total[c.UPDATED_OFFENSE_CATEGORY == "Narcotics"].iloc[0]
prop_ddpp